#📌 Extracão

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def extract_telecom_data_github():

    print("Conectando à API da Telecom X...")

    try:
        url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json'
        }

        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        data = response.json()

        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data.get('customers', data))

        print(f"{len(df)} registros extraídos com sucesso!")
        return df

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
        return None


#🔧 Transformação

In [4]:
def handle_missing_values(df):
    df_clean = df.copy()

    if 'gender' in df_clean.columns and df_clean['gender'].isnull().any():
        mode_gender = df_clean['gender'].mode()[0]
        df_clean['gender'] = df_clean['gender'].fillna(mode_gender)

    if 'InternetService' in df_clean.columns and df_clean['InternetService'].isnull().any():
        df_clean['InternetService'] = df_clean['InternetService'].fillna('No internet service')

    if 'TotalCharges' in df_clean.columns:
        df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
        mask_new_customers = (df_clean['TotalCharges'].isnull()) & (df_clean['tenure'] <= 1)
        df_clean.loc[mask_new_customers, 'TotalCharges'] = df_clean.loc[mask_new_customers, 'MonthlyCharges']
        remaining_nulls = df_clean['TotalCharges'].isnull()
        if remaining_nulls.any():
            df_clean['TotalCharges'] = df_clean['TotalCharges'].fillna(df_clean['TotalCharges'].median())

    return df_clean

def remove_duplicates(df):
    df_dedup = df.copy()
    initial_count = len(df_dedup)
    df_dedup = df_dedup.drop_duplicates()

    if 'customerID' in df_dedup.columns:
        df_dedup = df_dedup.loc[df_dedup.groupby('customerID').apply(
            lambda x: x.isnull().sum(axis=1).idxmin(), include_groups=False
        )]

    return df_dedup

def fix_categorical_inconsistencies(df):
    df_consistent = df.copy()
    corrections = {
        'gender': {'M': 'Male', 'F': 'Female', 'male': 'Male', 'female': 'Female'},
        'Partner': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Dependents': {'Y': 'Yes', 'N': 'No', 'yes': 'Yes', 'no': 'No'},
        'Churn': {'1': 'Yes', '0': 'No', 'TRUE': 'Yes', 'FALSE': 'No'}
    }

    for col, mapping in corrections.items():
        if col in df_consistent.columns:
            df_consistent[col] = df_consistent[col].replace(mapping)

    return df_consistent

def add_daily_charges(df):
    df_new = df.copy()
    if 'MonthlyCharges' in df_new.columns:
        df_new['Contas_Diarias'] = df_new['MonthlyCharges'] / 30
    return df_new

def transform_binary_columns(df):
    df_transformed = df.copy()
    binary_map = {'Yes': 1, 'No': 0}

    binary_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

    for col in binary_columns:
        if col in df_transformed.columns:
            df_transformed[col] = df_transformed[col].map(binary_map)

    return df_transformed

def standardize_data_types(df):
    df_typed = df.copy()

    categorical_columns = ['gender', 'InternetService', 'Contract', 'PaymentMethod']
    for col in categorical_columns:
        if col in df_typed.columns:
            df_typed[col] = df_typed[col].astype('category')

    numeric_columns = {'MonthlyCharges': 'float64', 'TotalCharges': 'float64',
                      'tenure': 'int64', 'SeniorCitizen': 'int64'}

    for col, dtype in numeric_columns.items():
        if col in df_typed.columns:
            if col in ['MonthlyCharges', 'TotalCharges']:
                df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')
            df_typed[col] = df_typed[col].astype(dtype)

    return df_typed


In [5]:
def run_transformation_pipeline(df):
    df_clean = handle_missing_values(df)
    df_dedup = remove_duplicates(df_clean)
    df_consistent = fix_categorical_inconsistencies(df_dedup)
    df_daily = add_daily_charges(df_consistent)
    df_binary = transform_binary_columns(df_daily)
    df_final = standardize_data_types(df_binary)

    return df_final

#📊 Carga e análise

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_descriptive_statistics(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns

    stats_summary = {}
    stats_summary['numeric'] = df[numeric_cols].describe()
    stats_summary['categorical'] = df[categorical_cols].describe()

    return stats_summary

def analyze_churn_distribution(df):
    if 'Churn' not in df.columns:
        return None

    churn_counts = df['Churn'].value_counts()
    churn_percentages = df['Churn'].value_counts(normalize=True) * 100

    distribution_summary = {
        'counts': churn_counts,
        'percentages': churn_percentages
    }

    return distribution_summary

def analyze_categorical_vs_churn(df, categorical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in categorical_columns:
        if col in df.columns:
            crosstab = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
            results[col] = crosstab

    return results

def analyze_numerical_vs_churn(df, numerical_columns):
    if 'Churn' not in df.columns:
        return None

    results = {}

    for col in numerical_columns:
        if col in df.columns:
            group_stats = df.groupby('Churn')[col].agg(['mean', 'median', 'std', 'count'])
            results[col] = group_stats

    return results

def create_churn_visualizations(df):
    visualizations_data = {}

    if 'Churn' in df.columns:
        churn_counts = df['Churn'].value_counts()
        visualizations_data['churn_distribution'] = {
            'labels': churn_counts.index.tolist(),
            'values': churn_counts.values.tolist()
        }

    return visualizations_data

In [7]:
def generate_correlation_matrix(df):
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()
    return correlation_matrix

def run_analysis_pipeline(df):
    descriptive_stats = calculate_descriptive_statistics(df)
    churn_distribution = analyze_churn_distribution(df)

    categorical_cols = ['gender', 'Contract', 'PaymentMethod', 'InternetService']
    categorical_analysis = analyze_categorical_vs_churn(df, categorical_cols)

    numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Contas_Diarias']
    numerical_analysis = analyze_numerical_vs_churn(df, numerical_cols)

    correlations = generate_correlation_matrix(df)
    visualizations = create_churn_visualizations(df)

    analysis_results = {
        'descriptive_statistics': descriptive_stats,
        'churn_distribution': churn_distribution,
        'categorical_analysis': categorical_analysis,
        'numerical_analysis': numerical_analysis,
        'correlations': correlations,
        'visualizations': visualizations
    }

    return analysis_results


#📄Relatorio Final

In [8]:
def generate_introduction():
    introduction = {
        'objective': 'Analisar o comportamento de churn dos clientes da Telecom X',
        'problem': 'Alto índice de cancelamentos impacta a receita e crescimento da empresa',
        'goal': 'Identificar padrões e fatores que levam à evasão para desenvolver estratégias de retenção'
    }
    return introduction

def summarize_data_cleaning(original_records, final_records, issues_fixed):
    cleaning_summary = {
        'original_records': original_records,
        'final_records': final_records,
        'retention_rate': (final_records / original_records) * 100,
        'issues_addressed': issues_fixed
    }
    return cleaning_summary

def extract_key_insights(analysis_results):
    insights = []

    if 'churn_distribution' in analysis_results:
        churn_rate = analysis_results['churn_distribution']['percentages'].get(1, 0)
        insights.append(f"Taxa geral de churn: {churn_rate:.1f}%")

    if 'categorical_analysis' in analysis_results:
        contract_analysis = analysis_results['categorical_analysis'].get('Contract', None)
        if contract_analysis is not None:
            monthly_churn = contract_analysis.loc['Month-to-month', 1] if 'Month-to-month' in contract_analysis.index else 0
            insights.append(f"Contratos mensais apresentam maior risco de churn: {monthly_churn:.1f}%")

    if 'numerical_analysis' in analysis_results:
        tenure_analysis = analysis_results['numerical_analysis'].get('tenure', None)
        if tenure_analysis is not None:
            avg_tenure_churned = tenure_analysis.loc[1, 'mean']
            avg_tenure_retained = tenure_analysis.loc[0, 'mean']
            insights.append(f"Clientes que cancelaram têm tenure médio de {avg_tenure_churned:.1f} meses vs {avg_tenure_retained:.1f} meses dos que permaneceram")

    return insights

def generate_recommendations():
    recommendations = [
        'Implementar programa de fidelização para contratos de longo prazo',
        'Melhorar processo de onboarding para novos clientes',
        'Incentivar métodos de pagamento automáticos com descontos',
        'Criar campanhas direcionadas para clientes de alto risco',
        'Desenvolver sistema de alertas para identificação precoce de churn',
        'Oferecer benefícios escalonados baseados no tempo de permanência'
    ]
    return recommendations

def create_final_report(df_original, df_final, analysis_results):
    report = {}

    report['introducao'] = generate_introduction()

    report['limpeza_dados'] = summarize_data_cleaning(
        len(df_original),
        len(df_final),
        ['Valores ausentes tratados', 'Duplicatas removidas', 'Tipos padronizados', 'Inconsistências corrigidas']
    )

    report['analise_exploratoria'] = {
        'metodo': 'Análise descritiva, distribuição de churn, análise categórica e numérica',
        'ferramentas': 'Python, Pandas, visualizações estatísticas',
        'variaveis_analisadas': ['gender', 'tenure', 'Contract', 'MonthlyCharges', 'TotalCharges', 'PaymentMethod']
    }

    report['insights'] = extract_key_insights(analysis_results)

    report['conclusoes'] = [
        'Contratos mensais representam maior risco de churn',
        'Clientes novos são mais propensos ao cancelamento',
        'Método de pagamento influencia na retenção',
        'Tempo de permanência é inversamente correlacionado ao churn'
    ]

    report['recomendacoes'] = generate_recommendations()

    return report

In [10]:

def print_formatted_report(report):
    print("RELATÓRIO DE ANÁLISE DE CHURN - TELECOM X")
    print("="*50)

    print("\n1. INTRODUÇÃO")
    print("-"*20)
    intro = report['introducao']
    print(f"Objetivo: {intro['objective']}")
    print(f"Problema: {intro['problem']}")
    print(f"Meta: {intro['goal']}")

    print("\n2. LIMPEZA E TRATAMENTO DE DADOS")
    print("-"*35)
    cleaning = report['limpeza_dados']
    print(f"Registros originais: {cleaning['original_records']}")
    print(f"Registros finais: {cleaning['final_records']}")
    print(f"Taxa de retenção: {cleaning['retention_rate']:.1f}%")
    print("Correções aplicadas:")
    for issue in cleaning['issues_addressed']:
        print(f"  - {issue}")

    print("\n3. ANÁLISE EXPLORATÓRIA DE DADOS")
    print("-"*32)
    eda = report['analise_exploratoria']
    print(f"Método: {eda['metodo']}")
    print(f"Ferramentas: {eda['ferramentas']}")
    print(f"Variáveis analisadas: {', '.join(eda['variaveis_analisadas'])}")

    print("\n4. PRINCIPAIS INSIGHTS")
    print("-"*22)
    for insight in report['insights']:
        print(f"  - {insight}")

    print("\n5. CONCLUSÕES")
    print("-"*14)
    for conclusion in report['conclusoes']:
        print(f"  - {conclusion}")

    print("\n6. RECOMENDAÇÕES")
    print("-"*17)
    for recommendation in report['recomendacoes']:
        print(f"  - {recommendation}")

def run_final_report_pipeline(df_original, df_final, analysis_results):
    final_report = create_final_report(df_original, df_final, analysis_results)
    print_formatted_report(final_report)
    return final_report

